In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
git clone https://github.com/huggingface/neuralcoref.git

Cloning into 'neuralcoref'...
Checking out files: 100% (151/151), done.


In [ ]:
# !git clone https://github.com/huggingface/neuralcoref.git
!pip install https://huggingface.co/spacy/en_core_web_sm/resolve/main/en_core_web_sm-any-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 1.7 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.1
    Uninstalling en-core-web-sm-3.4.1:
      Successfully uninstalled en-core-web-sm-3.4.1


In [ ]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd neuralcoref

!pip install -r requirements.txt
!pip install -e .

/content/neuralcoref
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 5.3 MB/s 
     |████████████████████████████████| 208 kB 45.6 MB/s 
     |████████████████████████████████| 1.0 MB 49.5 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.3
    Uninstalling spacy-3.4.3:
      Successfully uninstalled spacy-3.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are ins

In [ ]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 2.8 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047104 sha256=e9787ced82403c5912ae78f61ad315ed4b9662e7e197334c66941e74654f9635
  Stored in directory: /root/.cache/pip/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.0
    Uninstalling en-core-web-sm-3.4.0:
      Successfully uninstalled en-core-web-sm-3.4.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

## Downloading the data

We'll be working with an English-to-Spanish translation dataset
provided by [Anki](https://www.manythings.org/anki/). Let's download it:

In [ ]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

2638744/2638744 [==============================] - 0s 0us/step


In [ ]:
import spacy
from spacy import displacy

# https://spacy.io/usage/linguistic-features
# https://spacy.io/usage/visualizers

nlp = spacy.load('en_core_web_sm')

In [ ]:
doc = nlp("A mistake young people often make is to start learning too many languages at the same time, as they underestimate the difficulties and overestimate their own ability to learn them.")

In [ ]:
displacy.render(doc, style="dep", jupyter=True)

In [ ]:
with open(text_file) as f:
  text_data = f.read()

In [ ]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en_core_web_sm')

# load NeuralCoref and add it to the pipe of SpaCy's model
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

# You're done. You can now use NeuralCoref the same way you usually manipulate a SpaCy document and it's annotations.

100%|██████████| 40155833/40155833 [00:00<00:00, 48796122.80B/s]


## Parsing the data

Each line contains an English sentence and its corresponding Spanish sentence.
The English sentence is the *source sequence* and Spanish one is the *target sequence*.
We prepend the token `"[start]"` and we append the token `"[end]"` to the Spanish sentence.

In [ ]:
feminine_markers = {'Ella', 'ella', 'Ellas', 'ellas', 'Mary', 'mary'}
masculine_markers = {'Él', 'él', 'Ellos', 'ellos', 'Tom', 'tom', 'Leo', 'leo'}

In [ ]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

In [ ]:
text_pairs = []

counter = 0
for line in lines:
    eng, spa = line.split("\t")
    spa_set = set(spa.split())

    doc = nlp(eng)
    engList = eng.split()
    coreferences = doc._.coref_clusters

    newEng = eng

    corefLookup = {} # all objects -> original subject

    for coref in coreferences:
      corefSubject, corefObjects = coref.mentions[0], set(coref.mentions[1:])

      for obj in corefObjects:
        corefLookup[obj] = corefSubject
        newEng = newEng.replace(" " + str(obj) + " ", " [" + str(corefSubject) + "] " + str(obj) + " ")
        newEng = newEng.replace(" " + str(obj) + ".", " [" + str(corefSubject) + "] " + str(obj) + ".")
    
    adjEng = newEng
    for word in doc:
      auxWords = {"is", "are", "was", "were"}
      adjAlreadyAdded = set()
      if word.pos_ == "NOUN" or word.pos_ == "PRON":
        children = [child for child in word.children]

        for child in children:
          if child.pos_ == "ADJ" and child.text not in adjAlreadyAdded:
            adjAlreadyAdded.add(child.text)
            adjObject = corefLookup[word.text] if word.text in corefLookup else word.text
            adjEng = adjEng.replace(" " + str(child.text) + " ", " [" + str(adjObject) + "] " + str(child.text) + " ")
            adjEng = adjEng.replace(" " + str(child.text) + ".", " [" + str(adjObject) + "] " + str(child.text) + ".")

      elif word.text in auxWords:
        children = [child for child in word.children]
        
        adjective = []
        nounPronoun = []

        for child in children:
          if child.pos_ == "ADJ":
            adjective.append(child)
          if child.pos_ == "NOUN" or child.pos_ == "PRON":
            nounPronoun.append(child)

        if adjective and nounPronoun and adjective[0].text not in adjAlreadyAdded:
          adjAlreadyAdded.add(adjective[0].text)
          adjObject = corefLookup[nounPronoun[0].text] if nounPronoun[0].text in corefLookup else nounPronoun[0].text
          adjEng = adjEng.replace(" " + str(adjective[0].text) + " ", " [" + str(adjObject) + "] " + str(adjective[0].text) + " ")
          adjEng = adjEng.replace(" " + str(adjective[0].text) + ".", " [" + str(adjObject) + "] " + str(adjective[0].text) + ".")

    eng = adjEng

    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))
    counter += 1

    with open('coreferenceAdjectiveResultSpanish.txt', 'w') as g:
      g.write(eng + "\t" + spa + "\n")

    if counter % 10000 == 0:
      print("Processed: ", counter)

Processed:  10000
Processed:  20000
Processed:  30000
Processed:  40000
Processed:  50000
Processed:  60000
Processed:  70000
Processed:  80000
Processed:  90000
Processed:  100000
Processed:  110000


In [ ]:
# print(f"Feminine ratio = {feminine_counter/(feminine_counter + masculine_counter)}")

Here's what our sentence pairs look like:

In [ ]:
for _ in range(30):
    print(random.choice(text_pairs))

('Hand in your papers.', '[start] Entrega tus papeles. [end]')
("We can't leave any [ends] loose ends.", '[start] No podemos dejar cabos sueltos. [end]')
("She's a [kisser] good kisser.", '[start] Ella es buena besadora. [end]')
("I don't want you seeing Tom anymore.", '[start] No quiero que veas más a Tom. [end]')
('I studied hard so that I could pass the examination.', '[start] Estudié mucho, de modo que pude aprobar el examen. [end]')
('He is a [champion] great swimming champion.', '[start] Él es un gran campeón de natación. [end]')
('Who will play the role of the princess?', '[start] ¿Quién hará el rol de la princesa? [end]')
('He broke the window on purpose.', '[start] Él rompió la ventana a propósito. [end]')
("Tom isn't interested either.", '[start] Tom no está interesado tampoco. [end]')
('I found the diary that my father kept for 30 years.', '[start] Encontré el diario que mi padre mantuvo por 30 años. [end]')
('It is [It] better to live rich, than to die rich.', '[start] Es m

Now, let's split the sentence pairs into a training set, a validation set,
and a test set.

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


## Vectorizing the text data

We'll use two instances of the `TextVectorization` layer to vectorize the text
data (one for English and one for Spanish),
that is to say, to turn the original strings into integer sequences
where each integer represents the index of a word in a vocabulary.

The English layer will use the default string standardization (strip punctuation characters)
and splitting scheme (split on whitespace), while
the Spanish layer will use a custom standardization, where we add the character
`"¿"` to the set of punctuation characters to be stripped.

Note: in a production-grade machine translation model, I would not recommend
stripping the punctuation characters in either language. Instead, I would recommend turning
each punctuation character into its own token,
which you could achieve by providing a custom `split` function to the `TextVectorization` layer.

In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

Next, we'll format our datasets.

At each training step, the model will seek to predict target words N+1 (and beyond)
using the source sentence and the target words 0 to N.

As such, the training dataset will yield a tuple `(inputs, targets)`, where:

- `inputs` is a dictionary with the keys `encoder_inputs` and `decoder_inputs`.
`encoder_inputs` is the vectorized source sentence and `encoder_inputs` is the target sentence "so far",
that is to say, the words 0 to N used to predict word N+1 (and beyond) in the target sentence.
- `target` is the target sentence offset by one step:
it provides the next words in the target sentence -- what the model will try to predict.

In [ ]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return ({"encoder_inputs": eng, "decoder_inputs": spa[:, :-1],}, spa[:, 1:])


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 20 steps long):

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


## Building the model

Our sequence-to-sequence Transformer consists of a `TransformerEncoder`
and a `TransformerDecoder` chained together. To make the model aware of word order,
we also use a `PositionalEmbedding` layer.

The source sequence will be pass to the `TransformerEncoder`,
which will produce a new representation of it.
This new representation will then be passed
to the `TransformerDecoder`, together with the target sequence so far (target words 0 to N).
The `TransformerDecoder` will then seek to predict the next words in the target sequence (N+1 and beyond).

A key detail that makes this possible is causal masking
(see method `get_causal_attention_mask()` on the `TransformerDecoder`).
The `TransformerDecoder` sees the entire sequences at once, and thus we must make
sure that it only uses information from target tokens 0 to N when predicting token N+1
(otherwise, it could use information from the future, which would
result in a model that cannot be used at inference time).


In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config

In [ ]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
epochs = 3  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   3155456     ['positional_embedding[

In [ ]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(200):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence, translated)

The thief ran away. [start] el fuego se [UNK] [end]
I've decided to do that by myself. [start] he hecho solo que me he hecho [end]
My wife loves apple pie. [start] a mi esposa le encanta la puerta [end]
Can I reserve a flight to Chicago? [start] puedo comprar un par de semanas [end]
She lays the table for breakfast. [start] ella está en la mesa [end]
I hope we achieve that. [start] espero que nos [UNK] eso [end]
Somebody took away my bag. [start] alguien se puso mi hija [end]
Did you miss me? [start] me has hecho [end]
The period is missing at the end of the sentence. [start] el tiempo se ha [UNK] el béisbol [end]
I'm thinking of going to the States during the summer vacation. [start] estoy pensando en el verano para los verano [end]
Tom asked Mary if [Tom] he could read [Mary] her mother's letter. [start] tom le preguntó a mary si pudo ser su madre [end]
You can't drink seawater because it is too [it] salty. [start] no puedes comer porque es tan bueno como es [end]
He had trouble brea

In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
translated_sentences = []
for i in range(100):
  input_sentence = test_eng_texts[i]
  translated = decode_sequence(input_sentence)
  translated_sentences.append(translated)

test_sp_texts = [pair[1] for pair in test_pairs]

bleu_scores = []
for j in range(len(translated_sentences)):
  bleu_scores.append(sentence_bleu(translated_sentences[j], test_sp_texts[j], weights = (1, 0, 0, 0)))
  print(translated_sentences[j], test_sp_texts[j] )
print(bleu_scores) 
print(np.sum(np.array(bleu_scores))/len(bleu_scores))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

[start] yo he ido a la guerra [end] [start] Yo estudié antes de la cena. [end]
[start] no te [UNK] [end] [start] No te encontrarán. [end]
[start] [UNK] a tu inglés [end] [start] Trata de mejorar tu inglés. [end]
[start] has oído el sofá [end] [start] ¿Viste el debate presidencial? [end]
[start] algunas personas se puede un poco de agua [end] [start] Algo de sal proviene de las minas, y algo del agua. [end]
[start] es la mismo tiempo [end] [start] Siempre es lo mismo. [end]
[start] el rojo está lejos de la calle [end] [start] Sudáfrica está lejos. [end]
[start] la reunión fue [UNK] [end] [start] La reunión fue cancelada. [end]
[start] no quiero que te [UNK] [end] [start] No quiero molestarte. [end]
[start] creo que es demasiado tarde para decir [end] [start] Supongo que es muy tarde para decir que no. [end]
[start] nadie vino a ayudar [end] [start] No vino nadie a ayudarme. [end]
[start] tom se puso anoche [end] [start] Tom se emborrachó anoche. [end]
[start] dónde está la oficina de dó

In [ ]:
for i in range(len(test_eng_texts)):
    input_sentence = test_eng_texts[i]
    translated = decode_sequence(input_sentence)

    with open('translatedResult.txt', 'a') as h:
      h.write(input_sentence + "\t" + translated + "\n")